# Entrenamiento - Tracking y Testeo

**Importe de librerias**

In [0]:
#modelos
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
#evaluación
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#estructura
from pyspark.ml.functions import vector_to_array
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
#tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#mlflow
import mlflow
import mlflow.spark
from mlflow.tracking import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, ColSpec




#numpy
import numpy as np




In [0]:
%python
#Instalación de libreria para optimizar hiperparametros con buenas practicas
#optimizar la selección de hiperparametros
%pip install hyperopt
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Carga de Datos

Estos datos fueron generados en el Notebook "01_Preparacion_EDA", los cuales ya se encuentran transformados listos para utilizar en el entrenamiento

In [0]:
df_churn_T = spark.read.format("delta").load("/Volumes/corona/silver/transform")
display(df_churn_T.limit(5))

features,Churn
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0"",""0.0"",""1.0"",""0.0"",""0.495243504188556"",""0.09683373562402385"",""0.34374556297032516"",""0.4966633536845094"",""0.3448814425670879"",""0.43944341899758627"",""0.4931137299446259"",""0.3989777083629135"",""0.39542808462303"",""0.5501916796819537"",""0.3357944057929859""]}",0
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""1.0"",""0.0"",""0.0"",""0.504756495811444"",""0.48132897912821243"",""0.34374556297032516"",""0.2866676132329973"",""0.43844952435041884"",""0.3438875479199205"",""0.4931137299446259"",""0.3989777083629135"",""0.39542808462303"",""0.20914383075394008"",""0.22887973874769274""]}",0
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""1.0"",""1.0"",""1.0"",""0.504756495811444"",""0.48132897912821243"",""0.34374556297032516"",""0.2866676132329973"",""0.3448814425670879"",""0.43944341899758627"",""0.4931137299446259"",""0.3989777083629135"",""0.39542808462303"",""0.5501916796819537"",""0.22887973874769274""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""0.0"",""0.0"",""0.0"",""0.504756495811444"",""0.09683373562402385"",""0.34374556297032516"",""0.2866676132329973"",""0.43844952435041884"",""0.3438875479199205"",""0.2902172369728809"",""0.3989777083629135"",""0.39542808462303"",""0.20914383075394008"",""0.21922476217520942""]}",0
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""1.0"",""1.0"",""1.0"",""0.495243504188556"",""0.48132897912821243"",""0.4395854039471816"",""0.4966633536845094"",""0.43844952435041884"",""0.43944341899758627"",""0.4931137299446259"",""0.3989777083629135"",""0.39542808462303"",""0.5501916796819537"",""0.3357944057929859""]}",1


## Separación de datos - entrenamiento y evaluación

In [0]:
#Voy a utilizar el 80% para el entrenamiento y 20% para evaluar el modelo
train_df, test_df = df_churn_T.randomSplit([0.8, 0.2], seed=42)
print(f"Train: {train_df.count()} - Test: {test_df.count()}")

Train: 5634 - Test: 1409


## Experimento Mlflow

tracking

In [0]:
mlflow.set_experiment("/Users/josestunt95@gmail.com/ChurnPrediction")

2025/11/23 17:14:15 INFO mlflow.tracking.fluent: Experiment with name '/Users/josestunt95@gmail.com/ChurnPrediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2671492619073786', creation_time=1763918055262, experiment_id='2671492619073786', last_update_time=1763918055262, lifecycle_stage='active', name='/Users/josestunt95@gmail.com/ChurnPrediction', tags={'mlflow.experiment.sourceName': '/Users/josestunt95@gmail.com/ChurnPrediction',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'josestunt95@gmail.com',
 'mlflow.ownerId': '8415925107621832'}>

## Regresión Logistica

In [0]:
#Calculo de metricas
def compute_metrics(preds):
    """
    Calcula ACC, F1 y AUC usando Spark SQL.
    """

    preds = preds.withColumn("prediction_int", preds["prediction"].cast("int"))

    # Accuracy
    acc = preds.filter(preds.Churn == preds.prediction_int).count() / preds.count()

    # F1 Score
    tp = preds.filter("prediction_int = 1 AND Churn = 1").count()
    fp = preds.filter("prediction_int = 1 AND Churn = 0").count()
    fn = preds.filter("prediction_int = 0 AND Churn = 1").count()

    precision = tp / (tp + fp + 1e-9)
    recall = tp / (tp + fn + 1e-9)
    f1 = 2 * ((precision * recall) / (precision + recall + 1e-9))

    #AUC
    auc = evaluator.evaluate(preds)

    return auc, acc, f1

In [0]:
#Asignación de variable de entorno apuntar a volumen para guardar modelo
VOLUME_TMP = "/Volumes/corona/gold/models/ml_tmp"

#Función para pasar el vector a un array
def vector_to_columns(df, vector_col="features"):
    #convertir vector -> array<double>
    df = df.withColumn(f"{vector_col}_arr", vector_to_array(vector_col))

    #obtener tamaño
    first = df.select(f"{vector_col}_arr").head()
    size = len(first[f"{vector_col}_arr"])

    #crear columnas primitivas
    for i in range(size):
        df = df.withColumn(f"{vector_col}_{i}", df[f"{vector_col}_arr"][i])

    return df, size


def build_signature_and_example(preds):
    """
    Convierte features vector en array para el signature. esto es netamente para el registro, ya que evita errores con el registro de vector de predicciones
    """
    
    preds_cols, vec_size = vector_to_columns(preds, "features")

    input_columns = [f"features_{i}" for i in range(vec_size)]

    input_example = preds_cols.select(input_columns).limit(1).toPandas().to_dict(orient="records")[0]

    input_schema = Schema([ColSpec("double", col) for col in input_columns])
    output_schema = Schema([ColSpec("double")])

    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    return signature, input_example

#Función objetivo de regresión logistica, 
def objectivo_lr(params):

    with mlflow.start_run(run_name="LR_Hyperopt"):

        lr = LogisticRegression(
            featuresCol="features",
            labelCol="Churn",
            regParam=params["regParam"],
            elasticNetParam=params["elasticNetParam"]
        )

        model = lr.fit(train_df)
        preds = model.transform(test_df)

        auc, acc, f1 = compute_metrics(preds)

        signature, input_example = build_signature_and_example(preds)

        mlflow.log_params(params)
        mlflow.log_metric("AUC", auc)
        mlflow.log_metric("Accuracy", acc)
        mlflow.log_metric("F1", f1)

        mlflow.spark.log_model(
            model,
            artifact_path="LR_Model",
            signature=signature,
            input_example=input_example,
            dfs_tmpdir=VOLUME_TMP
        )

    return {"loss": -auc, "status": STATUS_OK}

#Hiperparametros opcionales
space_lr = {
    "regParam": hp.choice("regParam", [0.0, 0.001, 0.01, 0.1, 0.3]),
    "elasticNetParam": hp.uniform("elasticNetParam", 0.0, 1.0)
}

trials_lr = Trials()

#Aplicación y registro de entrenamiento de modelo con Hyperopt
best_lr = fmin(
    fn=objectivo_lr,
    space=space_lr,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials_lr
)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

2025/11/23 19:02:11 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2025/11/23 19:02:13 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-4fa5dc13-0dd2-4ab5-8c0f-4e/tmpv_2c6ndj/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 

2025/11/23 19:02:14 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "features_0": 0.0,
    "features_1": 0.0,
    "features_2": 0.0,
    "features_3": 0.0,
    "features_4": 0.0,
    "features_5": 0.495243504188556,

Uploading artifacts:   0%|          | 0/22 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:16<05:06, 16.15s/trial, best loss: -1.0]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000590 seconds

INFO:hyperopt.tpe:TPE using 1/1 trials with best loss -1.000000

2025/11/23 19:02:25 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2025/11/23 19:02:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-4fa5dc13-0dd2-4ab5-8c0f-4e/tmp6s9hl5hd/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 

2025/11/23 19:02:28 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "features_0": 0.0

...

Uploading artifacts:   0%|          | 0/22 [00:00<?, ?it/s]

 95%|█████████▌| 19/20 [04:30<00:14, 14.00s/trial, best loss: -1.0]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000590 seconds

INFO:hyperopt.tpe:TPE using 19/19 trials with best loss -1.000000

2025/11/23 19:06:39 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2025/11/23 19:06:41 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-4fa5dc13-0dd2-4ab5-8c0f-4e/tmpfv33mf18/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 

2025/11/23 19:06:41 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "features_0": 0

Uploading artifacts:   0%|          | 0/22 [00:00<?, ?it/s]

100%|██████████| 20/20 [04:43<00:00, 14.18s/trial, best loss: -1.0]


In [0]:
print("------------> Mejores hiperparámetros LR son:", best_lr)

INFO:py4j.clientserver:Received command c on object id p0



------------> Mejores hiperparámetros LR son: {'elasticNetParam': np.float64(0.36828278457007546), 'regParam': np.int64(0)}


## Selección y Registro del Modelo - Regresión Logistica

In [0]:
#Consulta MLflow para seleccionar la mejor LR por AUC como base
best_lr_run_info = mlflow.search_runs(
filter_string="tags.mlflow.runName = 'LR_Hyperopt'",
order_by=["metrics.AUC DESC"],
max_results=1
).iloc[0]


best_lr_run_id = best_lr_run_info.run_id
lr_model_uri = f"runs:/{best_lr_run_id}/LR_Model"


lr_registered = mlflow.register_model(
model_uri=lr_model_uri,
name="Churn_LR_Model"
)


print("Mejor modelo de regresión logistica registrado:", lr_registered)

Successfully registered model 'workspace.default.churn_lr_model'.


Uploading artifacts:   0%|          | 0/22 [00:00<?, ?it/s]

INFO:py4j.clientserver:Closing down clientserver connection

INFO:py4j.clientserver:Closing down clientserver connection

INFO:py4j.clientserver:Closing down clientserver connection

INFO:py4j.clientserver:Closing down clientserver connection

INFO:py4j.clientserver:Closing down clientserver connection

INFO:py4j.clientserver:Closing down clientserver connection

INFO:py4j.clientserver:Closing down clientserver connection

INFO:py4j.clientserver:Closing down clientserver connection



✔ Logistic Regression best model registered: <ModelVersion: aliases=[], creation_timestamp=1763925166681, current_stage=None, description='', last_updated_timestamp=1763925169855, name='workspace.default.churn_lr_model', run_id='d6e63ea57d8e48eba395bccf23a3fb78', run_link=None, source='dbfs:/databricks/mlflow-tracking/2671492619073786/d6e63ea57d8e48eba395bccf23a3fb78/artifacts/LR_Model', status='READY', status_message='', tags={}, user_id='josestunt95@gmail.com', version='1'>


Created version '1' of model 'workspace.default.churn_lr_model'.


## Random Forest

In [0]:
#Función objetivo de aplicación de Random Forest
def objectivo_rf(params):
    with mlflow.start_run(run_name="RF_Hyperopt"):
        rf = RandomForestClassifier(
            featuresCol="features",
            labelCol="Churn",
            numTrees=int(params["numTrees"]),
            maxDepth=int(params["maxDepth"])
        )

        model = rf.fit(train_df)
        preds = model.transform(test_df)

        auc, acc, f1 = compute_metrics(preds)

        signature, input_example = build_signature_and_example(preds)

        mlflow.log_params(params)
        mlflow.log_metric("AUC", auc)
        mlflow.log_metric("Accuracy", acc)
        mlflow.log_metric("F1", f1)

        mlflow.spark.log_model(
            model,
            artifact_path="RF_Model",
            signature=signature,
            input_example=input_example,
            dfs_tmpdir=VOLUME_TMP
        )

    return {"loss": -auc, "status": STATUS_OK}

INFO:py4j.clientserver:Received command c on object id p0



In [0]:
#Opción de hiperparametros a probar en el modelo
space_rf = {
"numTrees": hp.choice("numTrees", [10, 20, 30, 40, 50]),
"maxDepth": hp.choice("maxDepth", [3, 5, 7, 10])
}

In [0]:
trials_rf = Trials()

#Aplicación y registro de entrenamiento de modelo con Hyperopt
best_rf = fmin(
fn=objectivo_rf,
space=space_rf,
algo=tpe.suggest,
max_evals=20,
trials=trials_rf
)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001139 seconds

INFO:hyperopt.tpe:TPE using 0 trials

2025/11/23 19:26:49 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2025/11/23 19:26:52 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-4fa5dc13-0dd2-4ab5-8c0f-4e/tmplawhr3ix/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 

2025/11/23 19:26:52 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "features_0": 0.0,
    "features_1": 0.0,
  

Uploading artifacts:   0%|          | 0/26 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:17<05:30, 17.40s/trial, best loss: -1.0]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000671 seconds

INFO:hyperopt.tpe:TPE using 1/1 trials with best loss -1.000000

2025/11/23 19:27:07 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2025/11/23 19:27:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-4fa5dc13-0dd2-4ab5-8c0f-4e/tmpdoh8io_t/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 

2025/11/23 19:27:09 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "features_0": 0.0

...

Uploading artifacts:   0%|          | 0/26 [00:00<?, ?it/s]

 95%|█████████▌| 19/20 [05:27<00:17, 17.19s/trial, best loss: -1.0]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000738 seconds

INFO:hyperopt.tpe:TPE using 19/19 trials with best loss -1.000000

2025/11/23 19:32:16 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2025/11/23 19:32:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-4fa5dc13-0dd2-4ab5-8c0f-4e/tmpexgug43w/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 

2025/11/23 19:32:18 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "features_0": 0

Uploading artifacts:   0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 20/20 [05:45<00:00, 17.28s/trial, best loss: -1.0]


## Selección y Registro de Random Forest

Se selecciona el modelo con mejor resultado en las ejecuciones con diferentes hiperparametros

In [0]:
#Consulta MLflow para seleccionar la mejor RF por AUC como base
best_rf_run_info = mlflow.search_runs(
filter_string="tags.mlflow.runName = 'RF_Hyperopt'",
order_by=["metrics.AUC DESC"],
max_results=1
).iloc[0]


best_rf_run_id = best_rf_run_info.run_id
rf_model_uri = f"runs:/{best_rf_run_id}/RF_Model"


rf_registered = mlflow.register_model(
model_uri=rf_model_uri,
name="Churn_RF_Model"
)


print("Random Forest model registrado:", rf_registered)

Successfully registered model 'workspace.default.churn_rf_model'.


Uploading artifacts:   0%|          | 0/26 [00:00<?, ?it/s]

✔ Random Forest best model registered: <ModelVersion: aliases=[], creation_timestamp=1763926346653, current_stage=None, description='', last_updated_timestamp=1763926350113, name='workspace.default.churn_rf_model', run_id='2fa6be48a44141cfa97005e3d38bc203', run_link=None, source='dbfs:/databricks/mlflow-tracking/2671492619073786/2fa6be48a44141cfa97005e3d38bc203/artifacts/RF_Model', status='READY', status_message='', tags={}, user_id='josestunt95@gmail.com', version='1'>


Created version '1' of model 'workspace.default.churn_rf_model'.


## Conclución

Se registran dos modelos, regresión logistica y Random forest. Por temas de tiempo no itere demasiadas ocasiones o revise mas en profundidad la perfección del modelo; tuve enfoque en construir el flujo de trabajo completo.

## Uso

In [0]:
import mlflow.spark

# Cargar modelos desde Model Registry
lr_model = mlflow.spark.load_model("models:/churn_lr_model/1")
rf_model = mlflow.spark.load_model("models:/churn_rf_model/1")


# Ejemplo de predicción
# predictions_lr = lr_model.transform(spark_df)
# predictions_rf = rf_model.transform(spark_df)


/databricks/python/lib/python3.12/site-packages/databricks/sdk/errors/base.py:87: UserWarning: The 'retry_after_secs' parameter of DatabricksError is deprecated and will be removed in a future version.
  warnings.warn(


---------------------------------------------------------------------------
MlflowException                           Traceback (most recent call last)
File <command-8963420621813008>, line 4
      1 import mlflow.spark
      3 # Cargar modelos desde Model Registry
----> 4 lr_model = mlflow.spark.load_model("models:/churn_lr_model/1")
      5 rf_model = mlflow.spark.load_model("models:/churn_rf_model/1")

File /databricks/python/lib/python3.12/site-packages/mlflow/spark/__init__.py:985, in load_model(model_uri, dfs_tmpdir, dst_path)
    983 sparkml_model_uri = append_to_uri_path(model_uri, flavor_conf["model_data"])
    984 local_sparkml_model_path = os.path.join(local_mlflow_model_path, flavor_conf["model_data"])
--> 985 return _load_model(
    986     model_uri=sparkml_model_uri,
    987     dfs_tmpdir_base=dfs_tmpdir,
    988     local_model_path=local_sparkml_model_path,
    989 )

File /databricks/python/lib/python3.12/site-packages/mlflow/spark/__init__.py:896, in _load_model(mod

In [0]:

#Cargar modelos desde Model Registry usando dfs_tmpdir en un volumen creado en el volumen corona en el shema gold, volumen models
lr_model = mlflow.spark.load_model(
    "models:/churn_lr_model/1",
    dfs_tmpdir="/Volumes/corona/gold/models/mlflow_tmp"
)
rf_model = mlflow.spark.load_model(
    "models:/churn_rf_model/1",
    dfs_tmpdir="/Volumes/corona/gold/models/mlflow_tmp"
)

# Ejemplo de predicción
# predictions_lr = lr_model.transform(spark_df)
# predictions_rf = rf_model.transform(spark_df)

/databricks/python/lib/python3.12/site-packages/databricks/sdk/errors/base.py:87: UserWarning: The 'retry_after_secs' parameter of DatabricksError is deprecated and will be removed in a future version.
  warnings.warn(


**Simular datos nuevos**

In [0]:
nuevo_cliente_features = [
    1.0,   #Partner (Yes)
    0.0,   #Dependents (No)
    1.0,   #PhoneService (Yes)
    1.0,   #PaperlessBilling (Yes)
    0.0,   #Churn placeholder (no se usa)
    0.49,  #gender_freq
    0.33,  #MultipleLines_freq
    0.41,  #InternetService_freq
    0.28,  #OnlineSecurity_freq
    0.43,  #OnlineBackup_freq
    0.35,  #DeviceProtection_freq
    0.48,  #TechSupport_freq
    0.40,  #StreamingTV_freq
    0.39,  #StreamingMovies_freq
    0.55,  #Contract_freq
    0.33   #PaymentMethod_freq
]

In [0]:

#vector de entrada de los nuevos datos
vector_input = Vectors.dense(nuevo_cliente_features)

#construcción del dataframe de prueba
df_inferencia = spark.createDataFrame(
    [Row(features=vector_input)]
)

In [0]:
pred_lr = lr_model.transform(df_inferencia)
print("---------> Regresión Logistica")
pred_lr.select("prediction", "probability").show(truncate=False)

pred_rf = rf_model.transform(df_inferencia)
print("---------> Random Forest")
pred_rf.select("prediction", "probability").show(truncate=False)


---------> Regresión Logistica
+----------+----------------------------------------+
|prediction|probability                             |
+----------+----------------------------------------+
|0.0       |[0.8322375068575825,0.16776249314241753]|
+----------+----------------------------------------+

---------> Random Forest
+----------+----------------------------------------+
|prediction|probability                             |
+----------+----------------------------------------+
|0.0       |[0.9166511469139784,0.08334885308602145]|
+----------+----------------------------------------+

